In [63]:
import pandas as pd

1. načtení souboru
2. náhled hlavičky

In [64]:
properties = pd.read_csv('Nemovitosti.csv', encoding='UTF-8')
properties.head(1)

,Created At,Updated At,Scraped At,UID,ID,Source,Updated,Name,URL,Item Type,...,Ask Price Per Useful Area,Ask Price Per Living Area,Ask Price Fair Split,Days Since Published,Days On Display,Count Distinct Ask Price,Min Ask Price,Max Ask Price,First Ask Price,Delta Ask Price
0,2023-09-07T03:09:16.686831,2023-09-07T03:09:16.686832,2023-09-07T03:00:01,bezrealitky-206888,206888,bezrealitky,False,"Pronájem bytu 1+1 44 m², Antala Staška, Brno, ...",https://www.bezrealitky.cz/nemovitosti-byty-do...,Flat,...,329.545455,NaN,NaN,31,4,1,14500.0,14500.0,14500.0,0.0


Kontrola hodnot ve sloupci State:
1. zobrazení unikátních hodnot
2. nahrazení hodnot (NEW, VERY GOOD, GOOD, BAD, UNDEFINED)


In [65]:
properties['Real State'].unique()

array([ 2., nan,  1.,  3.,  4.])

In [66]:
properties['Real State'] = properties['Real State'].replace({1.0: 'NEW', 2.0: 'VERY GOOD', 3.0: 'GOOD', 4.0: 'BAD', float('nan'): 'UNDEFINED'})

Ze sloupce Address potřebujeme získat název městské části:
1. 

In [78]:
properties['Address'] = properties['Address'].str.replace('Jihomoravský kraj', '')
properties['Address'] = properties['Address'].str.rstrip(', ',)
properties['Address']


0       Antala Staška, Brno - Černá Pole
1             Karlova, Brno - Maloměřice
2              Veselá, Brno - Město Brno
3              Azurová, Brno - Řečkovice
4       Jungmannova, Brno - Královo Pole
                      ...               
2211           Wollmanova, Brno - Bystrc
2212               Vlněna, Brno - Trnitá
2213         Obřanská, Brno - Maloměřice
2214          Zaječí hora, Brno - Sadová
2215           Těšany, okres Brno-venkov
Name: Address, Length: 2216, dtype: object

In [84]:
properties[['Street', 'City part']] = properties['Address'].str.split(',', expand=True)
properties['City part'] = properties['City part'].str.strip()


In [85]:

properties['City part'].unique()

array(['Brno - Černá Pole', 'Brno - Maloměřice', 'Brno - Město Brno',
       'Brno - Řečkovice', 'Brno - Královo Pole', 'Brno - Štýřice',
       'Brno - Staré Brno', 'Brno - Bystrc', 'Brno - Zábrdovice',
       'Brno - Veveří', 'Brno - Stránice', 'Brno - Židenice',
       'Brno - Horní Heršpice', 'Brno - Starý Lískovec', 'Brno - Chrlice',
       'Brno - Líšeň', None, 'Brno - Husovice', 'Brno - Tuřany',
       'Brno - Komín', 'Brno - Medlánky', 'Brno - Žabovřesky',
       'Brno - Soběšice', 'Brno - Lesná', 'Brno - Dolní Heršpice',
       'Brno - Kohoutovice', 'Brno - Sadová', 'Město Brno',
       'Brno - Ořešín', 'Brno - Brno-Zábrdovice', 'Brno - Ivanovice',
       'Staré Brno', 'Brno - Bohunice', 'Brno - Ponava', 'Brno - Pisárky',
       'Brno - Černovice', 'Brno - Trnitá', 'Brno - Slatina',
       'Brno - Holásky', 'Brno - Žebětín', 'Brno - Obřany',
       'Brno - Kníničky', 'Brno - Brno-Štýřice', 'Brno - Nový Lískovec',
       'Brno - Komárov', 'Brno - Brno-Staré Brno', 'Brno - Jundr

Brno - Pisárky jsou rozdělené do dvou různých částí - střed a Kohoutovice, je třeba rozdělit ručně

In [108]:
properties['City part'] = properties['City part'].replace({
    'Brno - Staré Brno': 'Brno - střed', 
    'Brno - Veveří': 'Brno - střed', 
    'Brno - Štýřice': 'Brno - střed', 
    'Brno - Stránice': 'Brno - střed',  
    'Brno - Horní Heršpice': 'Brno - Jih', 
    'Brno - Soběšice': 'Brno - sever', 
    'Brno - Dolní Heršpice': 'Brno - jih', 
    'Brno - Sadová': 'Brno - Královo Pole', 
    'Brno - Brno-Zábrdovice': 'Brno - Zábrdovice', 
    'Staré Brno': 'Brno - střed', 
    'Brno - Ponava': 'Brno - Královo Pole', 
    'Brno - Trnitá': 'Brno - střed', 
    'Brno - Holásky': 'Brno - Tuřany', 
    'Brno - Obřany': 'Brno - Maloměřice a Obřany', 
    'Brno - Brno-Štýřice': 'Brno - Štýřice', 
    'Brno - Přízřenice': 'Brno - jih',
    'Brno - Brno-sever': 'Brno - sever',
    'Brno - Brněnské Ivanovice': 'Brno - Tuřany',
    'Brno - Brno-střed': 'Brno - střed',
    'Brno - Brno-Řečkovice a Mokrá Hora': 'Brno - Řečkovice a Mokrá Hora',
    'Brno - Brno-Židenice': 'Brno - Židenice',
    'Brno - Brno-jih': 'Brno - jih',
    'Brno - Brno-Královo Pole': 'Brno - Královo Pole',
    'Brno - Brno-Starý Lískovec': 'Brno - Starý Lískovec',
    'Město Brno': 'Brno - střed',
    'Brno - Město Brno': 'Brno - střed',
    'Brno - Brno-Staré Brno': 'Brno - střed',
    'Brno - Brno-město': 'Brno - střed',
    'okres Brno-venkov': 'Brno - venkov'})

properties['City part'].unique()

array(['Brno - Černá Pole', 'Brno - Maloměřice', 'Brno - střed',
       'Brno - Řečkovice', 'Brno - Královo Pole', 'Brno - Bystrc',
       'Brno - Zábrdovice', 'Brno - Židenice', 'Brno - Jih',
       'Brno - Starý Lískovec', 'Brno - Chrlice', 'Brno - Líšeň', None,
       'Brno - Husovice', 'Brno - Tuřany', 'Brno - Komín',
       'Brno - Medlánky', 'Brno - Žabovřesky', 'Brno - sever',
       'Brno - Lesná', 'Brno - jih', 'Brno - Kohoutovice',
       'Brno - Ořešín', 'Brno - Ivanovice', 'Brno - Bohunice',
       'Brno - Pisárky', 'Brno - Černovice', 'Brno - Slatina',
       'Brno - Žebětín', 'Brno - Maloměřice a Obřany', 'Brno - Kníničky',
       'Brno - Nový Lískovec', 'Brno - Komárov', 'Brno - Jundrov',
       'okres Brno-město', 'Brno - venkov', 'Brno', 'Brno - Jehnice',
       'Brno - Řečkovice a Mokrá Hora', 'Brno - Bosonohy'], dtype=object)

In [130]:
properties.loc[properties['Street'] == 'Kamechy', 'City part'] = 'Brno - střed'
properties.loc[properties['Street'] == 'Purkyňova', 'City part'] = 'Brno - Medlánky'
properties.loc[properties['Street'] == 'Veveří', 'City part'] = 'Brno - střed'
properties.loc[properties['Street'] == 'Lipová', 'City part'] = 'Brno - střed'
properties.loc[properties['Street'] == 'Vídeňská', 'City part'] = 'Brno - jih'
properties.loc[properties['Street'] == 'Palackého třída', 'City part'] = 'Brno - Královo Pole'
properties.loc[properties['Street'] == 'Vranovská', 'City part'] = 'Brno - sever'
properties.loc[properties['Street'] == 'Černovická', 'City part'] = 'Brno - Černovice'
properties.loc[properties['Street'] == 'Botanická', 'City part'] = 'Brno - střed'
properties.loc[properties['Street'] == 'Černovičky', 'City part'] = 'Brno - Slatina'


In [131]:
Result = properties.loc[properties['City part'] == 'Brno', ['City part', 'Street', 'Address',]]
Result

,City part,Street,Address
530,Brno,Kigginsova,"Kigginsova, Brno"
554,Brno,Miry Figarové,"Miry Figarové, Brno"
579,Brno,Veslařská,"Veslařská, Brno"
880,Brno,Žižkova,"Žižkova, Brno"
1042,Brno,Tuřanka,"Tuřanka, Brno"
1054,Brno,Rumiště,"Rumiště, Brno"
1356,Brno,Ondřeje Sekory,"Ondřeje Sekory, Brno"
1454,Brno,Hrnčířská,"Hrnčířská, Brno"
1475,Brno,Kaštanová,"Kaštanová, Brno"
1563,Brno,Šumavská,"Šumavská, Brno"


Zápis upraveného souboru.

In [ ]:
properties.to_csv('Nemovitosti_v4.tsv', sep='\t', index=False)